# Критерии выбора моделей и методы отбора признаков
## Пример выбора архитектуры нейронной сети

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from matplotlib import gridspec
from matplotlib.image import imread
from mlxtend.plotting import plot_decision_regions
from mpl_toolkits import mplot3d
from scipy.special import softmax
from scipy.spatial.distance import cdist
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.datasets import make_classification, load_boston
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from sklearn.model_selection import KFold, ParameterGrid, train_test_split, LeaveOneOut
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

### MNIST Dataset

In [ ]:
MNIST_train = datasets.MNIST('./mnist', train=True, download=True, 
                             transform=transforms.ToTensor())

MNIST_test = datasets.MNIST('./mnist', train=False, download=True,
                            transform=transforms.ToTensor())

In [ ]:
fig, gs = plt.figure(figsize=(19,4)), gridspec.GridSpec(1, 4)

ax = []

for i in range(4):
    ax.append(fig.add_subplot(gs[i]))
    ax[i].imshow(np.array(MNIST_train[i][0][0]), 'gray')

plt.show()

### Полносвязная нейронная сеть 
![](fig2.png)

Перепишем в матричном виде:
$$
f(\mathbf{x}, \mathbf{W}) = \mathbf{W}_{n_2+1}\sigma\bigr( \cdots\mathbf{W}_2\sigma\bigr(\mathbf{W}_{1}\mathbf{x}\bigr)\cdots\bigr)
$$

В данном примере введены следующие обозначения:
- число $n$ --- размерность пространства признаков (`input_dim`);
- число $n_1$ --- размерность скрытого слоя (`hidden_dim`);
- число $n_2$ --- количество скрытых слоев (`num_layers`);
- число $n_3$ --- размерность пространства ответов (`output_dim`).

Заметим, что при $n_2=0$ получаем линейную модель, то есть линейная модель это частный случай полносвязного персептрона.

P.S. размерность скрытого слоя может зависить от номера слоя, для простоты рассмотрим фиксируемый размер скрытого слоя.

In [ ]:
class Perceptron(torch.nn.Module):
    def __init__(self, input_dim=784, num_layers=0, 
                 hidden_dim=64, output_dim=10, p=0.0, device='cpu'):
        super(Perceptron, self).__init__()
        
        self.layers = torch.nn.Sequential()
        
        prev_size = input_dim
        for i in range(num_layers):
            self.layers.add_module('layer{}'.format(i), 
                                  torch.nn.Linear(prev_size, hidden_dim))
            self.layers.add_module('relu{}'.format(i), torch.nn.ReLU())
            self.layers.add_module('dropout{}'.format(i), torch.nn.Dropout(p=p))
            prev_size = hidden_dim
        
        self.layers.add_module('classifier', 
                               torch.nn.Linear(prev_size, output_dim))        
        self.to(device)
        
    def forward(self, input):
        return self.layers(input)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = Perceptron(device=device)
model

In [ ]:
model = Perceptron(num_layers=1, device=device)
model

In [ ]:
model = Perceptron(num_layers=2, device=device)
model

In [ ]:
def testing(model, dataset):
    generator = torch.utils.data.DataLoader(dataset, batch_size=64)

    pred = []
    real = []
    for x, y in tqdm(generator, leave=False):
        x = x.view([-1, 784]).to(device)
        y = y.to(device)

        pred.extend(torch.argmax(model(x), dim=-1).cpu().numpy().tolist())
        real.extend(y.cpu().numpy().tolist())

    return np.mean(np.array(real) == np.array(pred)), \
           classification_report(real, pred)

In [ ]:
def trainer(model, dataset, loss_function, optimizer, epochs):
    for epoch in tqdm(range(epochs), leave=False):
        generator = torch.utils.data.DataLoader(dataset, batch_size=64, 
                                              shuffle=True)
        for x, y in tqdm(generator, leave=False):
            optimizer.zero_grad()
            x = x.view([-1, 784]).to(device)
            y = y.to(device)

            output = model(x)
            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()

In [ ]:
model = Perceptron(num_layers=0, device=device)

In [ ]:
_ = model.eval()
acc, report = testing(model, MNIST_test)
print(report)

In [ ]:
_ = model.train()
trainer(model=model, 
        dataset=MNIST_train, 
        loss_function=torch.nn.CrossEntropyLoss(), 
        optimizer=torch.optim.Adam(model.parameters(), lr=0.001), 
        epochs=4)

In [ ]:
_ = model.eval()
acc, report = testing(model, MNIST_test)
print(report)

#### Гиперпараметры, которые нужно подобрать
- num_layers
- hidden_dim
- lr
- p

Воспользуемся Cross Validation для их подбора.

In [ ]:
cross_val = KFold(3)
number_of_batch = cross_val.get_n_splits(MNIST_train)

grid = ParameterGrid({'num_layers': [0, 2], 
                      'hidden_dim': [8, 64],
                      'p': [0.3, 0.7],
                      'lr': [0.001]})

X_train = MNIST_train.transform(MNIST_train.data.numpy()).transpose(0,1)
Y_train = MNIST_train.targets.data

In [ ]:
scores = dict()
for item in tqdm(grid):
    list_of_scores = []
    for train_index, test_index in tqdm(cross_val.split(X_train), 
                                        total=number_of_batch, leave=False):
        x_train_fold = X_train[train_index]
        x_test_fold = X_train[test_index]
        y_train_fold = Y_train[train_index]
        y_test_fold = Y_train[test_index]

        traindata = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
        testdata = torch.utils.data.TensorDataset(x_test_fold, y_test_fold)


        model = Perceptron(num_layers=item['num_layers'], p=item['p'],
                           hidden_dim=item['hidden_dim'], device=device)
        _ = model.train()
        trainer(model=model, 
                dataset=traindata, 
                loss_function=torch.nn.CrossEntropyLoss(), 
                optimizer=torch.optim.Adam(model.parameters(), lr=item['lr']), 
                epochs=4)
        
        _ = model.eval()
        acc, report = testing(model, testdata)
        list_of_scores.append(acc)
    scores[str(item)] = [np.mean(list_of_scores)]
                

In [ ]:
def draw_table(data, title=['ACCURACY'], width=[60, 15]):    
    row_format = '|' + '|'.join([("{:>"+str(w)+"}") for w in width]) + '|'
    row_format_bet = '+' + '+'.join([("{:>"+str(w)+"}") for w in width]) + '+'
    
    print(row_format_bet.format(
        "-"*width[0], *["-"*width[i+1] for i, _ in enumerate(title)]))
    print(row_format.format("", *title))
    print(row_format_bet.format(
        "-"*width[0], *["-"*width[i+1] for i, _ in enumerate(title)]))
    for key in data:
        if len(key) > width[0]:
            row_name = '...' + key[len(key)-width[0]+3:]
        else:
            row_name = key
        print(row_format.format(row_name, *[round(x, 2) for x in data[key]]))
        print(row_format_bet.format(
            "-"*width[0], *["-"*width[i+1] for i, _ in enumerate(title)]))

In [ ]:
draw_table(scores)

## Оценка качества моделей

При выборе модели машиного обучения
$$
f: \mathbb{X} \to \mathbb{Y},
$$
модель выбирается согласно некоторому критерию $L$ (функции ошибки, минус логарифм правдоподобия и тд.). Обычно в качестве функции $L$ рассматривается некоторая функция ошибки модели $f$ на выборке $\mathfrak{D}$:
$$
f = \arg\min_{f \in \mathfrak{F}} L\bigr(f, \mathfrak{D}\bigr)
$$

В зависимости от вида функции $L$ разделяют два типа критериев:
1. внутрений критерий качества;
2. внешний критерий качества.

Далее будем рассматривать два типа выборок:
1. $\mathfrak{D}$ это вся выборка, которая доступна для выбора модели;
2. $\mathfrak{D}'$ это выборка на которой проверяется качество уже выбраной модели.
3. $\mathfrak{D}^{l_k}_k$ это $k$-я подвыборка выборки $\mathfrak{D}$ размера $l_k$.

### Внутрений критерий:
Простой пример для регрессии:
$$
f = \arg\min_{f\in \mathfrak{F}} \sum_{\left(x, y\right) \in \mathfrak{D}}\left(f(x) - y\right)^2
$$

### Внешний критерий:
1. Разделить выборку $\mathfrak{D}$ на две подвыборки
2. Leave One Out
3. Скользящий контроль
4. Бутсреп
5. Регуляризация
6. Критерий Акаике
7. BIC


### Пример переобучения

In [ ]:
np.random.seed(0)
l = 6
n = 1
w = np.random.randn(n)
X_tr = np.random.randn(l, n)
y_tr = X_tr@w + np.random.randn(l)

X_vl = np.random.randn(l, n)
y_vl = X_vl@w + np.random.randn(l)

X_ts = np.random.randn(l, n)
y_ts = X_ts@w + np.random.randn(l)

In [ ]:
x_begin = -1.05
x_end = 2.5
X_polinom = np.hstack([X_tr**0, X_tr**1, X_tr**2, X_tr**3, X_tr**4, X_tr**5])
w_polinom = np.linalg.inv(X_polinom.T@X_polinom)@X_polinom.T@y_tr
y_polinom = list(map(lambda x: np.array([x**0, x**1, x**2, x**3, x**4, x**5])@w_polinom, 
                     np.linspace(x_begin, x_end)))


In [ ]:
plt.plot(X_tr, y_tr, 'o', label = 'points train')
plt.plot(X_ts, y_ts, 'o', label = 'points test')
plt.plot(np.linspace(x_begin, x_end), w*np.linspace(x_begin, x_end), '-', label = 'real')
plt.plot(np.linspace(x_begin, x_end), y_polinom, '-', label = 'polinom')

plt.legend(loc='best')
plt.show()

## Отбор признаков

Используется два основных подхода
1. Генерация признаков.
2. Отбор существующих признаков.

### Генерация признаков

1. Построение статистик на основе уже существующих признаков и тд.
2. Выше мы использовали PCA, что тоже генерит новые признаки.
3. Нейросеть кроме последнего слоя, также можно рассматривать как метод генерации нового признаково пространства (к примеру прошлое домашнее задание)

### Отбор существующих признаков

#### Полный перебор

В данном случае берем множество всех подмножеств признакового описания.

In [ ]:
data = load_boston()

D_all = data['data'], data['target']
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(D_all[0], D_all[1], 
                                                    test_size=300)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
indexes = list(itertools.product([0, 1], repeat=13))

scores_train = dict()
scores_test = dict()
for i, ind in enumerate(tqdm(indexes)):
    ind = np.array(ind, dtype=bool)

    X_train_low = X_train[:, ind]
    X_test_low = X_test[:, ind]

    w = np.linalg.inv(X_train_low.T@X_train_low)@X_train_low.T@y_train

    scores_train[i] = np.mean((X_train_low@w - y_train)**2)
    scores_test[i] = np.mean((X_test_low@w - y_test)**2)
    

In [ ]:
best_train = sorted(scores_train, key=lambda x: scores_train[x])[0]
best_test = sorted(scores_test, key=lambda x: scores_test[x])[0]

In [ ]:
print('best for train')
print(indexes[best_train])
print(data['feature_names'][np.array(indexes[best_train], dtype=bool)].tolist())

print('best for test')
print(indexes[best_test])
print(data['feature_names'][np.array(indexes[best_test], dtype=bool)].tolist())

In [ ]:
print(data['DESCR'][:1227])

In [ ]:
scores = []

for i, ind in enumerate(indexes):
    scores.append((sum(ind), scores_train[i]))

scores = np.array(scores)


In [ ]:
fig = plt.figure(figsize=(15, 5))
scores = np.zeros([len(indexes), 2])
for i, ind in enumerate(indexes):
    scores[i] = [sum(ind), scores_train[i]]
plt.plot(scores[:, 0], scores[:, 1], 'ob')
plt.axhline(y=np.min(scores[:, 1]), color='b', linestyle='-', label = 'train')

for i, ind in enumerate(indexes):
    scores[i] = [sum(ind), scores_test[i]]
plt.plot(scores[:, 0], scores[:, 1], '.g')
plt.axhline(y=np.min(scores[:, 1]), color='g', linestyle='-', label = 'test')

plt.legend(loc='best'), plt.ylim((20, 30))
plt.show()

#### Жадный алгоритм: add

Жадно добавляем один признак, который дает максимальный прирост качества.

In [ ]:
np.random.seed(0)
X_val_train, X_val_test, y_val_train, y_val_test = train_test_split(X_train, y_train, test_size=50)


In [ ]:
J_star, J, current, k_star, d = [], set(), 99999999999, 0, 1
for k in range(X_val_train.shape[1]):
    scores_val_test = dict()
    for j in list(set(range(X_val_train.shape[1])) - J):
        ind = [ i in (J | {j}) for i in range(X_val_train.shape[1])]
        X_val_train_val = X_val_train[:, ind]
        X_val_test_val = X_val_test[:, ind]
        w = np.linalg.inv(X_val_train_val.T@X_val_train_val)@X_val_train_val.T@y_val_train
        scores_val_test[j] = np.mean((X_val_test_val@w - y_val_test)**2)

    best = sorted(scores_val_test, key=lambda x: scores_val_test[x])[0]

    J.add(best)
    
    if scores_val_test[best] < current:
        current = scores_val_test[best]
        k_star = k
        J_star = set(J)
    if k - k_star > d:
        break
ind = np.array([ i in J_star for i in range(X_val_train.shape[1])])

In [ ]:
print('best for train')
print(np.array(indexes[best_train], dtype=int))
print(data['feature_names'][np.array(indexes[best_train], dtype=bool)].tolist())

print('best for validation')
print(np.array(ind, dtype=int))
print(data['feature_names'][np.array(ind, dtype=bool)].tolist())

print('best for test')
print(np.array(indexes[best_test], dtype=int))
print(data['feature_names'][np.array(indexes[best_test], dtype=bool)].tolist())

#### Жадный алгоритм: add-del

Реализовать в качестве домашнего задания

## Оценка качества классификации

Основные функции для оценки качества классификации:
1. Accuracy (доля верных ответов)
2. Precision (доля релевантных среди всех найденных)
3. Recall (доля найденных среди релевантных)

### Задача:
Простая задача бинарной классификации. Прототип алгоритма:

Вход: признаковое описание объекта.

Выход: вероятность класса $1$. (и соответсвенно класс объекта на основе treshold)

Метрики качества:
$$
ACC = \frac{TP + TN}{TP + TN + FP + FN}
$$

$$
PRECISION = \frac{TP}{TP + FP}
$$

$$
RECALL = \frac{TP}{TP + FN}
$$

In [ ]:
X, Y = make_classification(n_samples=400, n_features=2, 
                           n_informative=2, n_classes=2, 
                           n_redundant=0,
                           n_clusters_per_class=1,
                           random_state=0)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=100, 
                                                    random_state=0)
X_train_val, X_test_val, Y_train_val, Y_test_val = train_test_split(
    X_train, Y_train, test_size=100, random_state=0)

In [ ]:
model = SVC(probability=True)
_ = model.fit(X_train_val, Y_train_val)

fpr, tpr, thresholds = roc_curve(Y_test_val, model.predict_proba(X_test_val)[:,1], pos_label=1)

In [ ]:
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = {})'.format(round(auc(fpr, tpr), 2)))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
plt.plot(thresholds, tpr, lw = 2, label = 'tpr')
plt.plot(thresholds, 1-fpr, lw = 2, label = '1-fpr')

threshold = thresholds[np.argmin((tpr - 1 + fpr)**2)]
plt.axvline(x=threshold, 
            ls='--', c='black',
            label='best threshold {}'.format(round(threshold, 2)))
plt.xlabel('threshold')
plt.legend(loc="lower right")
plt.show()

In [ ]:
print(classification_report(Y_test, model.predict_proba(X_test)[:, 1] > 0.5))

In [ ]:
print(classification_report(Y_test, model.predict_proba(X_test)[:, 1] > threshold))

## Как правильно составлять выборки

Большая часть времени занимает построение выборки, которую уже потом требуется аппроксимировать. Правильное составление выборки позволяет решить много проблем в дальнейшем.

Простые правила как правильно составить выборку:
1. Сразу определиться с объектом исследования и целевой переменной.
    * физичиский смысл признаков играет большую роль, так как именно они позволяют интерпретировать результаты модели, поэтому обязательно информацию о физическом смысле каждого признака требется сохранить.
    * если рассматривается задача классификации, то требуется зафиксировать классы, описать эти классы, построить биекцию между классами и их названием физическим описанием.
2. Выполнить разделение выборки на обучение и контроль заранее, убедившись, что они не пересекаются
    * проверить что выборки статистически не различаються.
    * для задачи классификации проверить, что баланс классов в обучении и контроле совпадает.